<a href="https://colab.research.google.com/github/solver-Mart1n/data-science/blob/solver-Mart1n-c05w1p2-FinalProject/reference/ibm_ds/language/python/final_project/answers/Final%20Assignment%20-%20Jupyter%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0220ENSkillsNetwork900-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


<h1>Extracting and Visualizing Stock Data</h1>
<h2>Description</h2>


Extracting essential data from a dataset and displaying it is a necessary part of data science; therefore individuals can make correct decisions based on the data. In this assignment, you will extract some stock data, you will then display this data in a graph.


<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Define a Function that Makes a Graph</li>
        <li>Question 1: Use yfinance to Extract Stock Data</li>
        <li>Question 2: Use Webscraping to Extract Tesla Revenue Data</li>
        <li>Question 3: Use yfinance to Extract Stock Data</li>
        <li>Question 4: Use Webscraping to Extract GME Revenue Data</li>
        <li>Question 5: Plot Tesla Stock Graph</li>
        <li>Question 6: Plot GameStop Stock Graph</li>
    </ul>
<p>
    Estimated Time Needed: <strong>30 min</strong></p>
</div>

<hr>


***Note***:- If you are working in IBM Cloud Watson Studio, please replace the command for installing nbformat from `!pip install nbformat==4.2.0` to simply `!pip install nbformat`


In [53]:
!pip install yfinance==0.1.67
#!mamba install bs4==4.10.0 -y
!pip install bs4 #==4.10.0 # -y
!pip install nbformat #==4.2.0

# pandas version used in Final_Assignment_Library.ipynb
!pip install pandas==1.3.3

In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In Python, you can ignore warnings using the warnings module. You can use the filterwarnings function to filter or ignore specific warning messages or categories.


In [54]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Define Graphing Function


In this section, we define the function `make_graph`. You don't have to know how the function works, you should only care about the inputs. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.


In [55]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Question 1: Use yfinance to Extract Stock Data


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.


In [56]:
tesla = yf.Ticker("TSLA")
tesla.info

{'regularMarketPrice': None, 'preMarketPrice': None, 'logo_url': ''}

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `tesla_data`. Set the `period` parameter to `max` so we get information for the maximum amount of time.


In [57]:
tesla_data = tesla.history(period="max") # The TSLA stock ticker is sensitive to the pandas version, e.g., 1.3.3, is used here

**Reset the index** using the `reset_index(inplace=True)` function on the tesla_data DataFrame and display the first five rows of the `tesla_data` dataframe using the `head` function. Take a screenshot of the results and code from the beginning of Question 1 to the results below.


In [58]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,281494500,0,0.0
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,257806500,0,0.0
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,123282000,0,0.0
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,77097000,0,0.0
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,103003500,0,0.0


## Question 2: Use Webscraping to Extract Tesla Revenue Data


Use the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm Save the text of the response as a variable named `html_data`.


In [59]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
html_data  = requests.get(url).text

Parse the html data using `beautiful_soup`.


In [60]:
soup = BeautifulSoup(html_data, 'html5lib')

Using `BeautifulSoup` or the `read_html` function extract the table with `Tesla Revenue` and store it into a dataframe named `tesla_revenue`. The dataframe should have columns `Date` and `Revenue`.


<details><summary>Click here if you need help locating the table</summary>

```
    
Below is the code to isolate the table, you will now need to loop through the rows and columns like in the previous lab
    
soup.find_all("tbody")[1]
    
If you want to use the read_html function the table is located at index 1

We are focusing on quarterly revenue in the lab.
```

</details>


In [61]:
tables = soup.find_all("table")
print("There are",len(tables),"tables.")

for i,table in enumerate(tables):
    if("evenue" in str(table)):
        print(i,table.th.text)

There are 6 tables.
0 Tesla Annual Revenue(Millions of US $)
1 Tesla Quarterly Revenue(Millions of US $)
2 Sector
3 Stock Name
4 Link Preview
5 Link Preview


In [62]:
print(tables[0].prettify()) # shows Tesla Annual Revenue

<table class="historical_data_table table">
 <thead>
  <tr>
   <th colspan="2" style="text-align:center">
    Tesla Annual Revenue
    <br/>
    <span style="font-size:14px;">
     (Millions of US $)
    </span>
   </th>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td style="text-align:center">
    2021
   </td>
   <td style="text-align:center">
    $53,823
   </td>
  </tr>
  <tr>
   <td style="text-align:center">
    2020
   </td>
   <td style="text-align:center">
    $31,536
   </td>
  </tr>
  <tr>
   <td style="text-align:center">
    2019
   </td>
   <td style="text-align:center">
    $24,578
   </td>
  </tr>
  <tr>
   <td style="text-align:center">
    2018
   </td>
   <td style="text-align:center">
    $21,461
   </td>
  </tr>
  <tr>
   <td style="text-align:center">
    2017
   </td>
   <td style="text-align:center">
    $11,759
   </td>
  </tr>
  <tr>
   <td style="text-align:center">
    2016
   </td>
   <td style="text-align:center">
    $7,000
   </td>
  </tr>
  <tr>
   <td style="

In [63]:
print(tables[1].prettify()) # shows Tesla Quarterly Revenue

<table class="historical_data_table table">
 <thead>
  <tr>
   <th colspan="2" style="text-align:center">
    Tesla Quarterly Revenue
    <br/>
    <span style="font-size:14px;">
     (Millions of US $)
    </span>
   </th>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td style="text-align:center">
    2022-09-30
   </td>
   <td style="text-align:center">
    $21,454
   </td>
  </tr>
  <tr>
   <td style="text-align:center">
    2022-06-30
   </td>
   <td style="text-align:center">
    $16,934
   </td>
  </tr>
  <tr>
   <td style="text-align:center">
    2022-03-31
   </td>
   <td style="text-align:center">
    $18,756
   </td>
  </tr>
  <tr>
   <td style="text-align:center">
    2021-12-31
   </td>
   <td style="text-align:center">
    $17,719
   </td>
  </tr>
  <tr>
   <td style="text-align:center">
    2021-09-30
   </td>
   <td style="text-align:center">
    $13,757
   </td>
  </tr>
  <tr>
   <td style="text-align:center">
    2021-06-30
   </td>
   <td style="text-align:center">
    $11,95

In [64]:
revenue_data = pd.DataFrame(columns=["Date", "Revenue"])

Using the bs4 method, revenue data can be extracted as follows using the index 1, which corresponds to the Quarterly Revenue data as stated in the hint.

In [65]:
table_index = 1 # As mentioned in the hint, use the Quarterly Revenue data
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        revenue = col[1].text

        revenue_data = revenue_data.append({"Date":date, "Revenue":revenue}, ignore_index=True)
revenue_data

,Date,Revenue
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"
5,2021-06-30,"$11,958"
6,2021-03-31,"$10,389"
7,2020-12-31,"$10,744"
8,2020-09-30,"$8,771"
9,2020-06-30,"$6,036"


Without the hint, the same approach will have to be applied to index 0 as well to derive all revenue related tables.

In [66]:
revenue_data_annual = pd.DataFrame(columns=["Date", "Tesla Annual Revenue(Millions of US $)"])
table_index = 0 # As mentioned in the hint, use the Quarterly Revenue data
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        revenue = col[1].text

        revenue_data_annual = revenue_data_annual.append({"Date":date, "Tesla Annual Revenue(Millions of US $)":revenue}, ignore_index=True)
revenue_data_annual

,Date,Tesla Annual Revenue(Millions of US $)
0,2021,"$53,823"
1,2020,"$31,536"
2,2019,"$24,578"
3,2018,"$21,461"
4,2017,"$11,759"
5,2016,"$7,000"
6,2015,"$4,046"
7,2014,"$3,198"
8,2013,"$2,013"
9,2012,$413


Alternatively, using *read_html()*, revenue_data can be extracted from index [1], which is the Quarterly Revenue.

In [67]:
revenue_data_rh = pd.read_html(url, match="evenue",flavor='bs4')[1]
revenue_data_rh.columns = ['Date','Tesla Quarterly Revenue(Millions of US $)']
revenue_data_rh

,Date,Tesla Quarterly Revenue(Millions of US $)
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"
5,2021-06-30,"$11,958"
6,2021-03-31,"$10,389"
7,2020-12-31,"$10,744"
8,2020-09-30,"$8,771"
9,2020-06-30,"$6,036"


The same approach applies for the Annual Revenue table, index[0].

In [68]:
revenue_data_annual_rh = pd.read_html(url, match="evenue",flavor='bs4')[0]
revenue_data_annual_rh.columns = ['Date','Tesla Annual Revenue(Millions of US $)']
revenue_data_annual_rh

,Date,Tesla Annual Revenue(Millions of US $)
0,2021,"$53,823"
1,2020,"$31,536"
2,2019,"$24,578"
3,2018,"$21,461"
4,2017,"$11,759"
5,2016,"$7,000"
6,2015,"$4,046"
7,2014,"$3,198"
8,2013,"$2,013"
9,2012,$413


What about indices 2 to 5? Why are we skipping them?

'2' is a total revenue record.

In [69]:
revenue_data_2_rh = pd.read_html(url, match="evenue",flavor='bs4')[2]
revenue_data_2_rh

,Sector,Industry,Market Cap,Revenue
0,Auto/Tires/Trucks,Auto Manufacturers - Domestic,$549.575B,$53.823B
1,Tesla is the market leader in battery-powered ...,Tesla is the market leader in battery-powered ...,Tesla is the market leader in battery-powered ...,Tesla is the market leader in battery-powered ...


'3' and '4' are previews accessible through a URL.

In [70]:
revenue_data_3_rh = pd.read_html(url, match="evenue",flavor='bs4')[3]
revenue_data_3_rh

,Link Preview,HTML Code (Click to Copy)
0,Tesla Revenue 2010-2022 | TSLA,NaN
1,Macrotrends,NaN
2,Source,NaN


In [71]:
revenue_data_4_rh = pd.read_html(url, match="evenue",flavor='bs4')[4]
revenue_data_4_rh

,Link Preview,HTML Code (Click to Copy)
0,Tesla Revenue 2010-2022 | TSLA,NaN
1,Macrotrends,NaN
2,Source,NaN


'5' is apparently not a table.

In [72]:
revenue_data_5_rh = pd.read_html(url, match="evenue",flavor='bs4')[5]
revenue_data_5_rh

IndexError: list index out of range

Select which revenue data to use.

In [73]:
tesla_revenue = revenue_data # assigning specific version of revenue_data* to tesla_revenue

Execute the following line to remove the comma and dollar sign from the `Revenue` column.


In [74]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

Execute the following lines to remove an null or empty strings in the Revenue column.


In [75]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

Display the last 5 row of the `tesla_revenue` dataframe using the `tail` function. Take a screenshot of the results.


In [76]:
print(len(revenue_data))
print(len(tesla_revenue))
tesla_revenue.tail() # The discrepancy comes from removal of record 51 which is empty

54
53


,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


## Question 3: Use yfinance to Extract Stock Data


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is GameStop and its ticker symbol is `GME`.


In [77]:
game_stop = yf.Ticker("GME")
game_stop.info

{'regularMarketPrice': None, 'preMarketPrice': None, 'logo_url': ''}

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `gme_data`. Set the `period` parameter to `max` so we get information for the maximum amount of time.


In [78]:
gme_data = game_stop.history(period="max")

**Reset the index** using the `reset_index(inplace=True)` function on the gme_data DataFrame and display the first five rows of the `gme_data` dataframe using the `head` function. Take a screenshot of the results and code from the beginning of Question 3 to the results below.


In [79]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4: Use Webscraping to Extract GME Revenue Data


Use the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html. Save the text of the response as a variable named `html_data`.


In [80]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
html_data  = requests.get(url).text

Parse the html data using `beautiful_soup`.


In [81]:
soup = BeautifulSoup(html_data,'html5lib')

Using `BeautifulSoup` or the `read_html` function extract the table with `GameStop Revenue` and store it into a dataframe named `gme_revenue`. The dataframe should have columns `Date` and `Revenue`. Make sure the comma and dollar sign is removed from the `Revenue` column using a method similar to what you did in Question 2.


<details><summary>Click here if you need help locating the table</summary>

```
    
Below is the code to isolate the table, you will now need to loop through the rows and columns like in the previous lab
    
soup.find_all("tbody")[1]
    
If you want to use the read_html function the table is located at index 1


```

</details>


In [82]:
tables = soup.find_all("table")
print("There are",len(tables),"tables.")

for i,table in enumerate(tables):
    if("evenue" in str(table)):
        print(i,table.th.text)

There are 6 tables.
0 GameStop Annual Revenue(Millions of US $)
1 GameStop Quarterly Revenue(Millions of US $)
2 Sector
3 Stock Name
4 Link Preview
5 Link Preview


Display the last five rows of the `gme_revenue` dataframe using the `tail` function. Take a screenshot of the results.


First, select the html table for GameStop Quarterly Revenue(Millions of US $) with index [1]. Change the column variable names to *Date* and *Revenue*. Print the raw data frame length and the *head* as baseline.

In [83]:
gme_revenue = pd.read_html(url, match="evenue",flavor='bs4')[1]
gme_revenue.columns = ['Date','Revenue']

print("Data frame length before droping records with n/a value:",len(gme_revenue))
gme_revenue.head()

Data frame length before droping records with n/a value: 62


,Date,Revenue
0,2020-04-30,"$1,021"
1,2020-01-31,"$2,194"
2,2019-10-31,"$1,439"
3,2019-07-31,"$1,286"
4,2019-04-30,"$1,548"


Second, remove '$' and ',' symbols from the *Revenue* column and drop the records with n/a entries.

The data frame length will show a change in case records were dropped.

And finally, the data frame tail is displayed as per the requirement.

In [84]:
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")

gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

print("Data frame length after droping records with n/a value:",len(gme_revenue))

gme_revenue.tail()

Data frame length after droping records with n/a value: 62


,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


## Question 5: Plot Tesla Stock Graph


Use the `make_graph` function to graph the Tesla Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(tesla_data, tesla_revenue, 'Tesla')`. Note the graph will only show data upto June 2021.


## Question 6: Plot GameStop Stock Graph


Use the `make_graph` function to graph the GameStop Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(gme_data, gme_revenue, 'GameStop')`. Note the graph will only show data upto June 2021.


<h2>About the Authors:</h2>

<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

Azim Hirjani


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description        |
| ----------------- | ------- | ------------- | ------------------------- |
| 2024-05-04        | 1.2b     | Martin Borja | Added responses and extensions up to Question 4 |
| 2024-05-03        | 1.2a     | Martin Borja | Added responses and extensions up to Question 2 |
| 2022-02-28        | 1.2     | Lakshmi Holla | Changed the URL of GameStop |
| 2020-11-10        | 1.1     | Malika Singla | Deleted the Optional part |
| 2020-08-27        | 1.0     | Malika Singla | Added lab to GitLab       |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>

<p>
